Import the needed Python libraries

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

Read the data for soybean harvest to start doing the Exploratory Analysis

In [2]:
data = pd.read_csv("data.csv")


In [3]:
data.head(10)

,Season,Cultivar,Repetition,PH,IFP,NLP,NGP,NGL,NS,MHG,GY
0,1,NEO 760 CE,1,58.80,15.20,98.2,177.80,1.81,5.20,152.20,3232.82
1,1,NEO 760 CE,2,58.60,13.40,102.0,195.00,1.85,7.20,141.69,3517.36
2,1,NEO 760 CE,3,63.40,17.20,100.4,203.00,2.02,6.80,148.81,3391.46
3,1,NEO 760 CE,4,60.27,15.27,100.2,191.93,1.89,6.40,148.50,3312.58
4,1,MANU IPRO,1,81.20,18.00,98.8,173.00,1.75,7.40,145.59,3230.99
5,1,MANU IPRO,2,75.80,20.80,69.2,128.00,1.85,7.20,154.87,3374.80
6,1,MANU IPRO,3,84.40,15.80,95.4,161.80,1.70,6.80,150.23,3182.76
7,1,MANU IPRO,4,80.47,18.20,87.8,154.27,1.77,7.13,149.90,3165.72
8,1,77HO111I2X - GUAPORÉ,1,52.20,14.40,64.8,148.80,2.30,7.20,180.25,3640.46
9,1,77HO111I2X - GUAPORÉ,2,55.80,15.80,72.0,188.80,2.62,6.20,176.75,3602.34


Check the correlation between the variables so we can proceed to answer a question

In [4]:
data.corr()

ValueError: could not convert string to float: 'NEO 760 CE'

In this case we have choosen 2 variables with a strong correlation:

*   NLP. Number of Legumes per Plant
*   NGP. Number of grains per Plant

In [14]:
fig = px.scatter(data, x="NLP", y="NGP", color="Season",size='MHG', hover_data="Cultivar",trendline="ols", color_discrete_sequence=px.colors.qualitative.D3)
fig.show()